In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import pandas as pd
import shutil
import jupyter_notebooks.utils.aux_functions as aux
from scipy.special import expit
from tqdm import tqdm

from typing import List, Tuple

In [2]:
def sigmoid_correction(image, threshold, factor=10):
    normalized_image = image / 255.0

    corrected_image = expit(factor * (normalized_image - threshold))
    
    corrected_image = (corrected_image * 255).astype(np.uint8)
    
    return corrected_image

# Determinacion manual de umbrales Sigmoide

In [ ]:
path_image = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataset_features\0/20240612151612.jpg' # Sin sombras día
path_agua_mask = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mascara_agua_lescala_2.jpg'

factor = 8
threshold = 0.58
key = '  '
while key != ord('q'):
    mask_ag = cv2.imread(path_agua_mask)
    _, mask_agua_bin = cv2.threshold(mask_ag, 128, 1, cv2.THRESH_BINARY)
    mask_agua = np.uint8(mask_agua_bin)
    mask_agua = cv2.resize(mask_agua, (512, 288))
    
    img_rgb = cv2.imread(path_image)
    img_rgb = cv2.resize(img_rgb, (512, 288))
    
    img_rgb[:,:,0] = cv2.equalizeHist(img_rgb[:,:,0])
    img_rgb[:,:,1] = cv2.equalizeHist(img_rgb[:,:,1])
    img_rgb[:,:,2] = cv2.equalizeHist(img_rgb[:,:,2])
    
    res = np.zeros_like(img_rgb)
    
    res[:,:,0] = sigmoid_correction(img_rgb[:,:,0],threshold,factor)
    res[:,:,1] = sigmoid_correction(img_rgb[:,:,1],threshold,factor)
    res[:,:,2] = sigmoid_correction(img_rgb[:,:,2],threshold,factor)
    
    res = res*mask_agua
    img_gray = cv2.imread(path_image, cv2.IMREAD_GRAYSCALE)
    alto, ancho, canales = res.shape
    img_hsv = cv2.cvtColor(res, cv2.COLOR_BGR2HSV)
    
    imagenes = cv2.vconcat([res[:,:,0], res[:,:,1], res[:,:,2]])
    cv2.namedWindow('threshold: '+ str(threshold) + ' factor: ' + str(factor), cv2.WINDOW_NORMAL)
    cv2.moveWindow('threshold: '+ str(threshold) + ' factor: ' + str(factor), 0, 0)
    cv2.resizeWindow('threshold: '+ str(threshold) + ' factor: ' + str(factor), int(ancho), int(alto*3))
    cv2.imshow('threshold: '+ str(threshold) + ' factor: ' + str(factor), imagenes)
    
    key = cv2.waitKey(0)
    
    if key == ord('a'):
        threshold -= 0.02
        cv2.destroyAllWindows()
    
    if key == ord('s'): # subir limite inferior
        threshold += 0.02
        cv2.destroyAllWindows()
    
    if key == ord('d'):
        factor -= 1
        cv2.destroyAllWindows()
    
    if key == ord('f'): # subir limite inferior
        factor += 1
        cv2.destroyAllWindows()
        
    if key == ord('q'):  # sube limite inferior
        cv2.destroyAllWindows()

# Aplicación de corrección sigmoide

In [ ]:
path_images = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra'
path_save_img = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra\aplay_sigmoid'
factor = 8
threshold = 0.58

mask_ag = cv2.imread(path_agua_mask)
_, mask_agua_bin = cv2.threshold(mask_ag, 128, 1, cv2.THRESH_BINARY)
mask_agua = np.uint8(mask_agua_bin)
mask_agua = cv2.resize(mask_agua, (512, 288))

total = len(os.listdir(path_images))

for name_im in tqdm(os.listdir(path_images), desc="Procesando imágenes", total = total):
    im_path = os.path.join(path_images, name_im)
    img_rgb = cv2.imread(im_path)
    img_rgb = cv2.resize(img_rgb, (512, 288))
    
    img_rgb[:,:,0] = cv2.equalizeHist(img_rgb[:,:,0])
    img_rgb[:,:,1] = cv2.equalizeHist(img_rgb[:,:,1])
    img_rgb[:,:,2] = cv2.equalizeHist(img_rgb[:,:,2])
    
    res = np.zeros_like(img_rgb)
    
    res[:,:,0] = sigmoid_correction(img_rgb[:,:,0],threshold,factor)
    res[:,:,1] = sigmoid_correction(img_rgb[:,:,1],threshold,factor)
    res[:,:,2] = sigmoid_correction(img_rgb[:,:,2],threshold,factor)
    
    res = res*mask_agua
    
    cv2.imwrite(path_save_img + '/'+ name_im, res)

# Determinación de umbral intensidad y generación de máscara

In [6]:
path_img_sigTransform = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra\aplay_sigmoid'
path_orig = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra'
dir_destino = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra\mascaras'
almacen_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sin_sombra\procesadas'

transparencia = 0.4
kernel_dil = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

thresh = 180#100

for name_im in os.listdir(path_img_sigTransform):
    im_path = os.path.join(path_img_sigTransform, name_im)
    im_path_orig = os.path.join(path_orig, name_im)
    
    img = cv2.imread(im_path) # Shape: (2160, 3840, 3)
    img = cv2.resize(img, (512, 288)) # Shape: (512, 288, 3)
    img_orig = cv2.imread(im_path_orig)
    img_orig = cv2.resize(img_orig, (512, 288))
    
    key = '  '
    while (key != ord('q') and key != ord('g')):
        mask = np.ones_like(img[:,:,1], dtype=np.uint8)
        mask_out_of_range = (img[:,:,1] < thresh)
        mask[mask_out_of_range] = 0
        
        mask = cv2.dilate(mask, kernel_dil, iterations=1)
        
        #mask_rgb = cv2.cvtColor(mask*255, cv2.COLOR_GRAY2RGB)   Esta linea es para tener la máscara en color blanco
        mask_rgb = np.zeros_like(img) # Estas lineas son para tener la máscara en color rojo
        mask_rgb[:, :, 2] = mask * 255  # Estas lineas son para tener la máscara en color rojo
        
        composicion_pred = cv2.addWeighted(mask_rgb, transparencia, img_orig, 1 - transparencia, 0)
        imagenes = cv2.vconcat([img_orig, mask_rgb, composicion_pred])
        
        cv2.namedWindow(str(name_im[:-4]) + ' ' + str(thresh), cv2.WINDOW_NORMAL)
        cv2.moveWindow(str(name_im[:-4]) + ' ' + str(thresh), 3000, 0)
        cv2.resizeWindow(str(name_im[:-4]) + ' ' + str(thresh), int(img.shape[1]), int((img.shape[0])*3))
        cv2.imshow(str(name_im[:-4]) + ' ' + str(thresh), imagenes)
        
        key = cv2.waitKey(0)
    
        if key == ord('a'):
            thresh = thresh - 3
            cv2.destroyAllWindows()
        
        if key == ord('s'): # subir limite inferior
            thresh = thresh + 3
            cv2.destroyAllWindows()
        
        if key == ord('g'): # guardar máscara
            dir_save_mask = dir_destino + '/m_' + name_im
            cv2.imwrite(dir_save_mask, mask)
            shutil.move(im_path, almacen_procesadas)
            cv2.destroyAllWindows()
            
        if key == ord('q'):  # sube limite inferior
            cv2.destroyAllWindows()